## Start Inference Using the models

In [3]:
!pip install fasttext langid
!apt install g++
!pip install spacy
!python -m spacy download xx_ent_wiki_sm
!pip install sentencepiece
!pip install sacremoses
!pip install ctranslate2
!pip install langchain

UnboundLocalError: local variable 'child' referenced before assignment

349.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


## NLLB Running Local

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import torch
import pandas as pd
from datetime import datetime
import os

def get_latest_file(pattern):
    files = [f for f in os.listdir() if f.startswith(pattern)]
    return max(files, key=os.path.getmtime) if files else None

def load_checkpoint(latest_file, df, text_column):
    if latest_file:
        translated_df = pd.read_csv(latest_file)
        translated_texts = translated_df[text_column].tolist()
        start_idx = len(translated_texts)
    else:
        start_idx = 0
    return start_idx

def translate_batch(config, df):
    # Initialize model and tokenizer
    model_path = config["model_name"]
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    
    translator = pipeline(
        "translation",
        model=model,
        tokenizer=tokenizer,
        src_lang=config["src_lang"],
        tgt_lang=config["tgt_lang"],
        max_length=config["max_length"],
        device=config["device"]
    )

    # Load data and check starting index
    texts = df[config["text_column"]].tolist()
    start_idx = load_checkpoint(latest_file, df, config["text_column"]) if latest_file else 0
    print("Start From Index: ", start_idx)
    texts = df[config["text_column"]].to_list()
    
    # Determine output file name 
    prefix = config["prefix"]
    latest_file = get_latest_file(prefix)
    if latest_file:
        output_file = latest_file
    else:
        current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"{prefix}_{current_time}.csv"

    # Batch translation and saving
    batch_size = config["batch_size"]
    for i in tqdm(range(start_idx, len(texts), batch_size), desc="Translating", unit="batch"):
        batch = texts[i:i + batch_size]
        translated_batch = translator(batch)
        
        for j, text in enumerate(batch):
            updated_row = df.iloc[i + j].copy()
            updated_row["translated_text"] = translated_batch[j]["translation_text"]
            updated_dataframe = pd.DataFrame([updated_row])
            
            mode = "w" if i == start_idx and j == 0 and start_idx == 0 else "a"
            header = mode == "w"
            updated_dataframe.to_csv(output_file, index=False, mode=mode, header=header)

    print(f"Translation completed. Results saved to {output_file}")


# Unified configuration dictionary
config = {
    "model_name": "/home/lujun_li/projects/base_models/nllb-200-3.3B",
    "src_lang": "ltz_Latn",
    "tgt_lang": "eng_Latn",
    "device": 'cuda:0' if torch.cuda.is_available() else 'cpu',
    "max_length": 360,
    "batch_size": 5,
    "text_column": "subsentence",
    "prefix": "translation_nllb_"
}

input_file = "NC_lux_subsentences_test.csv"
dataset_df = pd.read_csv(input_file)
translate_batch(config)


## LLM running Local


In [ ]:
from langchain.prompts import PromptTemplate

# Define a function to generate the translation prompt
def generate_translation_prompt(text, language_1="English", language_2="Luxembourgish"):
    prompt_template = """Please translate the following {language_1} text into {language_2}. Please answer me with only translated text!

    ---------------------------------- Text to be translated ----------------------------------

    {Text}

    ---------------------------------- Text to be translated ----------------------------------

    """
    
    translation_prompt = PromptTemplate(
        input_variables=["language_1", "language_2", text],
        template=prompt_template
    )
    
    return translation_prompt.format(language_1=language_1, language_2=language_2, Text=text)



In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    GenerationConfig,
    pipeline,
)
from tqdm import tqdm
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os

def initialize_pipeline(model_config_dict):
    model_path = model_config_dict["CURRENT_RUNNING_MODEL"]
    if not model_path:
        raise ValueError("CURRENT_RUNNING_MODEL is not set")
    load_in_4bit, load_in_8bit = model_config_dict["CURRENT_LOAD_IN_4BIT"], model_config_dict["CURRENT_LOAD_IN_8BIT"]

    if model_config_dict["IF_LOADING_QUANTIZATION"]:
        nf4_config = BitsAndBytesConfig(load_in_4bit=load_in_4bit, load_in_8bit=load_in_8bit, bnb_4bit_compute_dtype=torch.float16)
    else:
        nf4_config = None
        
    generation_config = GenerationConfig.from_pretrained(model_path)
    generation_config.do_sample = True
    generation_config.max_length = 512
    generation_config.pad_token_id = 0
    generation_config.top_p = 0.9
    generation_config.temperature = 0.1
    generation_config.max_new_tokens = 512
    
    text_pipeline = pipeline("text-generation", model=model_path, torch_dtype=torch.float32, device_map="auto")
    # text_pipeline.model.generation_config = generation_config # This needs to write a blog on that
    text_pipeline.generation_config = generation_config
    return text_pipeline

def generate_text(pipeline, prompt):
    response = pipeline(prompt)[0]["generated_text"]
    return response

def find_most_recent_date(df, date_column):
    """Finds the most recent date in the specified date column of the DataFrame."""
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
    most_recent_date = df[date_column].max()
    return most_recent_date

def get_latest_file(pattern="translation_*.csv"):
    files = [f for f in os.listdir() if f.startswith("translation_LLM_") and f.endswith(".csv")]
    if not files:
        return None
    latest_file = max(files, key=os.path.getmtime)
    return latest_file

def load_checkpoint(latest_file, df, text_column="subsentence"):
    if latest_file:
        translated_df = pd.read_csv(latest_file)
        translated_texts = translated_df[text_column].tolist()
        start_idx = len(translated_texts)
    else:
        start_idx = 0
    return start_idx

def translate_batch_LLM(translator, df, text_column="subsentence", batch_size=1):

    df["prompts_inputs"] = df[text_column].apply(generate_translation_prompt)
    latest_file = get_latest_file()
    # Determine output file name based on the latest file
    if latest_file:
        output_file = latest_file
    else:
        current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"translation_LLM_{current_time}.csv"

    # Determine start index based on checkpoint file
    start_idx = load_checkpoint(latest_file, df, text_column) if latest_file else 0
    print ("Start From Index: ", start_idx)
    texts = df["prompts_inputs"].to_list()

    for i in tqdm(range(start_idx, len(texts), batch_size), desc="Translating", unit="batch"):
        batch = texts[i:i + batch_size]
        translated_batch = translator(batch)

        for j, text in enumerate(batch):
            updated_row = df.iloc[i + j].copy()
            updated_row["translated_text"] = translated_batch[j][0]['generated_text']
            updated_dataframe = pd.DataFrame([updated_row])
            
            if i == start_idx and j == 0 and not latest_file:
                updated_dataframe.to_csv(output_file, index=False, mode="w", header=True)
            else:
                updated_dataframe.to_csv(output_file, index=False, mode="a", header=False)

    print(f"Translation completed. Results saved to {output_file}")

model_config_dict = {
    "CURRENT_RUNNING_MODEL": "/home/lujun_li/projects/base_models/Llama-3.2-3B-Instruct",
    "IF_LOADING_QUANTIZATION": False,
    "CURRENT_LOAD_IN_4BIT":    True,
    "CURRENT_LOAD_IN_8BIT":    False
}

translator = initialize_pipeline(model_config_dict)
dataset_df = pd.read_csv("/home/lujun_li/projects/mt_luxembourgish/data/NC_lux/NC_lux_subsentences_test.csv")
translate_batch_LLM(translator = translator, df = dataset_df, text_column="subsentence", batch_size=1)

## LLM running with Ollama

In [1]:
from langchain.prompts import PromptTemplate

# Define a function to generate the translation prompt
def generate_translation_prompt(text, language_1 = "English", language_2 = "Luxembourgish"):
    prompt_template = """Please translate the following {language_1} text into {language_2}. Please answer me with only translated text!

    ---------------------------------- Text to be translated ----------------------------------

    {Text}

    ---------------------------------- Text to be translated ----------------------------------

    """
    
    translation_prompt = PromptTemplate(
        input_variables=["language_1", "language_2", text],
        template=prompt_template
    )
    
    return translation_prompt.format(language_1=language_1, language_2=language_2, Text=text)



In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import os
import requests
import json
import sys

project_root = "/Users/lujun.li/projects/mt_luxembourgish/"
if project_root not in sys.path:
    sys.path.append(project_root)


def generate_text_with_ollama(config, prompt):
    
    payload = {
        "model": config["model_name"],
        "prompt": prompt,
        "stream": False,
        "format": "json", # Without this, the api calling will be blocked
        "options": config["options"],
    }
    response = requests.post(config["server_url"], headers=config["headers"], data=json.dumps(payload))
    
    if response.status_code == 200:
        data = json.loads(response.text)
        actual_response = data["response"]
        return actual_response
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

def get_latest_file(prefix):
    files = [f for f in os.listdir() if f.startswith(prefix) and f.endswith(".csv")]
    if not files:
        return None
    latest_file = max(files, key=os.path.getmtime)
    return latest_file

def load_checkpoint(latest_file, df, text_column):
    if latest_file:
        translated_df = pd.read_csv(latest_file)
        translated_texts = translated_df[text_column].tolist()
        start_idx = len(translated_texts)
    else:
        start_idx = 0
    return start_idx

def translate_batch_ollama(config, df):
    df["prompts_inputs"] = df[config["text_column"]].apply(generate_translation_prompt)
    prefix = config["prefix"]
    latest_file = get_latest_file(prefix)
    
    if latest_file:
        output_file = latest_file
    else:
        current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"{prefix}_{current_time}.csv"

    start_idx = load_checkpoint(latest_file, df, config["text_column"]) if latest_file else 0
    print("Start From Index: ", start_idx)
    texts = df["prompts_inputs"].to_list()

    for i in tqdm(range(start_idx, len(texts), config["batch_size"]), desc="Translating", unit="batch"):
        batch = texts[i:i + config["batch_size"]]
        translated_batch = [generate_text_with_ollama(config, text) for text in batch]

        for j, text in enumerate(batch):
            updated_row = df.iloc[i + j].copy()
            updated_row["translated_text"] = translated_batch[j]
            updated_dataframe = pd.DataFrame([updated_row])
            
            if i == start_idx and j == 0 and not latest_file:
                updated_dataframe.to_csv(output_file, index=False, mode="w", header=True)
            else:
                updated_dataframe.to_csv(output_file, index=False, mode="a", header=False)

    print(f"Translation completed. Results saved to {output_file}")


config = {
    "model_name": "llama3.1:8b", # necessary
    "server_url": "http://localhost:11434/api/generate", # necessary
    "headers": {"Content-Type": "application/json"},  # necessary
    # model setting options
    "options": {
        "temperature": 0.1, # necessary
        "max_tokens": 512, # necessary
        "top_p": 0.9, # necessary
        "do_sample": True, # necessary
        "max_new_tokens": 512, # necessary
        "max_length": 512, # necessary
        "num_ctx": 2048, # necessary for acclelerating the translation process
    },
    "batch_size": 1, # use this to accelerate the translation process
    "prefix": "translation_LLM_ollama", # necessary
    "text_column": "subsentence", # necessary
}

input_file = "NC_lux_subsentences_test.csv"
dataset_df = pd.read_csv(input_file)
translate_batch_ollama(config=config, df=dataset_df)

# curl http://localhost:11434/api/generate -d '{
#   "model": "llama3.1:8b",
#   "prompt": "Please translate the following Luxembourgish text into English. Please answer me with only translated text! Dat léich ënnert anerem dorun, datt wärend der Coronakris d'Téléconsultatiounen agefouert goufen, vun deene méi jonk Leit méi profitéiert hätten.2020 hu ronn 6.700 Patiente vun enger Consultatioun bei engem Psychiater profitéiert - alles an allem ass et eng Hausse vun 2,1 Prozent. Wärend d'Zuel vu Patienten iwwert d'Joer gekuckt 2019 relativ stabel war, goufen et 2020 staark Variatiounen. D'Zuel vu Leit, déi an d'Consultatioun komm sinn, ass am 2.",
#   "format": "json",
#   "stream": false,
# }'


## LLM running with vllm